# Tracy, Jack and the wet grass

In [ ]:
from random import random as rand
flip = lambda theta: rand() < theta    # biased coin flip

In [ ]:
# simple implementation of the sprinkler network
p_R1      = 0.2            # p(R=1) = ...
p_S1      = 0.1            # p(S=1) = ...
p_J1_R1   = 1.0            # p(J=1|R=1) = ...
p_J1_R0   = 0.2            # p(J=1|R=0) = ...
p_T1_R1S0 = 1.0            # p(T=1|R=1,S=0) = ...
p_T1_R1S1 = 1.0            # p(T=1|R=1,S=1) = ...
p_T1_R0S1 = 0.9            # p(T=1|R=0,S=1) = ...
p_T1_R0S0 = 0.0            # p(T=1|R=0,S=0) = ...

In [ ]:
def sample():
    R = flip(p_R1)
    S = flip(p_S1)
    if R:
        J = flip(p_J1_R1)
    else:
        J = flip(p_J1_R0)
    if R and S:
        T = flip(p_T1_R1S1)
    elif R and not S:
        T = flip(p_T1_R1S0)
    elif not R and S:
        T = flip(p_T1_R0S1)            
    else:
        T = flip(p_T1_R0S0)
    return (R, S, J, T)

In [ ]:
def str(R, S, J, T):
    s = ''
    s += 'rain' if R else 'no rain'
    s += ', sprinkler' if S else ', no sprinkler'
    s += ', Jack' if J else ', no Jack'
    s += ', Tracy' if T else ', no Tracy'
    return s

In [ ]:
for i in range(50):
    print(str(*sample()))

In [ ]:
# calculate the joint probability
def joint_prob(R,S,J,T):
    p = 1.0
    p *= p_R1 if R else (1-p_R1)
    p *= p_S1 if S else (1-p_S1)
    if R:
        p *= p_J1_R1 if J else (1-p_J1_R1)
    else:
        p *= p_J1_R0 if J else (1-p_J1_R0)
    if R and S:
        p *= p_T1_R1S1 if T else (1-p_T1_R1S1)
    elif R and not S:
        p *= p_T1_R1S0 if T else (1-p_T1_R1S0)
    elif not R and S:
        p *= p_T1_R0S1 if T else (1-p_T1_R0S1)
    else:
        p *= p_T1_R0S0 if T else (1-p_T1_R0S0)
    return p
# test time
assert joint_prob(R=True, S=True, J=True, T=True) == p_R1*p_S1*p_J1_R1*p_T1_R1S1, "BUG"
assert joint_prob(R=True, S=False, J=True, T=True) == p_R1*(1-p_S1)*p_J1_R1*p_T1_R1S1, "BUG"
assert joint_prob(R=False, S=True, J=False, T=True) == (1-p_R1)*p_S1*(1-p_J1_R0)*p_T1_R0S1, "BUG"

In [ ]:
for i in range(20):
    print(joint_prob(*sample()))

In [ ]:
# calculate probabilities with the sum rule
def marginal_prob(R=None, S=None, J=None, T=None):
    # if the value is None, we sum it out!
    # note: 
    #    marginal_prob() == 1.0
    #    marginal_prob(R, S, J, T) == joint_prob(R, S, J, T)
    R = [True, False] if R is None else [R]  # must be iterable
    S = [True, False] if S is None else [S]  # must be iterable
    J = [True, False] if J is None else [J]  # must be iterable
    T = [True, False] if T is None else [T]  # must be iterable
    p = 0.0
    for r in R:
        for s in S:
            for j in J:
                for t in T:
                    p += joint_prob(r, s, j, t)
    return p
# test time:
assert marginal_prob() == 1.0, "BUG, probability must sum to one"
R, S, J, T = flip(0.5), flip(0.5), flip(0.5), flip(0.5)
m = marginal_prob(R, S, J, T)
j = joint_prob(R, S, J, T)
assert m == j, "BUG"

In [ ]:
# calculate conditional probability
def cond_prob(R=None, S=None, J=None, T=None,
             R_cond=None, S_cond=None, J_cond=None, T_cond=None):
    # the '*_cond' are the variable we condition on
    # note:
    #    cond_prob(R, S, J, T) == marginal_prob(R, S, J, T)
    #    cond_prob(R=True, R_cond=False) == 0.0
    denominator = marginal_prob(R_cond, S_cond, J_cond, T_cond)
    # let's collect the variables for the nominator
    if R_cond is None: R_nom = R         # keep the LHS
    elif    R is None: R_nom = R_cond    # keep the RHS
    elif  R == R_cond: R_nom = R         # keep any
    else: return 0.0                     # LHS and RHS missmatch
    if S_cond is None: S_nom = S
    elif    S is None: S_nom = S_cond
    elif  S == S_cond: S_nom = S
    else: return 0.0
    if J_cond is None: J_nom = J
    elif    J is None: J_nom = J_cond
    elif  J == J_cond: J_nom = J
    else: return 0.0
    if T_cond is None: T_nom = T
    elif    T is None: T_nom = T_cond
    elif  T == T_cond: T_nom = T
    else: return 0.0
    nominator = marginal_prob(R_nom, S_nom, J_nom, T_nom)
    return nominator/denominator
# test time
assert cond_prob(R, S, J, T) == marginal_prob(R, S, J, T), "BUG"
assert cond_prob(R=True, R_cond=False) == 0.0, "BUG"

In [ ]:
# surprisingly it also works with number 1 and 0!
# here are the examples from the lecture
print(f'p(S=1|T=1)      == {cond_prob(S=1, T_cond=1)}')
print(f'p(S=1|T=1,J=1)) == {cond_prob(S=1,T_cond=1,J_cond=1)}')